In [14]:
import os
import sys
import time
from datetime import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *

In [15]:
file_path = '/Users/yantingting/Seafile/风控/模型/13 菲律宾/日报&周报/运营指标监控'

In [16]:
filename = "运营指标_%s.xlsx" % datetime.strftime(datetime.now().date(), "%Y-%m-%d")
writer = pd.ExcelWriter(filename)

In [17]:
query1 = '''
select
date(a.created_at) apply_date,
case when a.product_name = 'CashNiJuan' then 'CNJ' else 'P2G&Jet' end as product_name,
a.order_no,a.order_type,
a.customer_phone_num phone,
a.application_status,
case when a.application_status in ('CreateApplication', 'InitialAuditingReject', 'AutoAuditing', 'AutoReject', 'RiskControlReject') then null else 1 end  JSTG,
case when a.application_status in ('AuditingPassed')then 1 else null end RSTG,
 b.loan_flow_status,b.loan_principal,c.actual_paid_off_date,c.due_date,c.FDQ,
d.DPD3,e.DPD7
from dw_flb_approval_phl_approval_prod_approval_applications a
LEFT JOIN
(select order_no, loan_principal, loan_flow_status from dw_flb_hx_phl_core_prod_core_loans where loan_flow_status='LoanSuccess') b
on a.order_no=b.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
case when (case when actual_paid_off_date = '1970-01-01'  then current_date -due_date else actual_paid_off_date-due_date end)>0 then 1 else 0 end as FDQ
from dw_flb_hx_phl_core_prod_core_repayment_schedules where extend_period=0
and due_date<=current_date - INTERVAL '1 day') c
on a.order_no = c.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
case when (case when actual_paid_off_date = '1970-01-01'  then current_date -due_date else actual_paid_off_date-due_date end)>3 then 1 else 0 end as DPD3
from dw_flb_hx_phl_core_prod_core_repayment_schedules where  extend_period=0
and due_date<=current_date - INTERVAL '3 day') d
on a.order_no = d.order_no
LEFT JOIN
(select order_no,actual_paid_off_date,due_date,
case when (case when actual_paid_off_date = '1970-01-01'  then current_date -due_date else actual_paid_off_date-due_date end)>7 then 1 else 0 end as DPD7
from dw_flb_hx_phl_core_prod_core_repayment_schedules where (product_name = 'Peso2Go' or product_name = 'JetPeso') and extend_period=0
and due_date<=current_date - INTERVAL '7 day') e
on a.order_no = e.order_no
where  date(a.deleted_at) = '1970-01-01'  and a.created_at>current_date - INTERVAL '60 day';
'''

In [18]:
df = data_con.database_gn(query1)
save_data_to_pickle(df,file_path, 'df.pkl')

Successful connection


In [20]:
df.head()

,apply_date,product_name,order_no,order_type,phone,application_status,jstg,rstg,loan_flow_status,loan_principal,actual_paid_off_date,due_date,fdq,dpd3,dpd7,新老标签
0,2019-11-24,P2G&Jet,Jp201911240391,NewApplicationOrder,9505159542,ApplicationCanceled,1.0,NaN,None,None,None,None,NaN,NaN,NaN,新客
1,2019-11-24,P2G&Jet,Jp201911240414,NewApplicationOrder,9663441136,AutoReject,NaN,NaN,None,None,None,None,NaN,NaN,NaN,新客
2,2019-11-24,P2G&Jet,Jp201911240431,NewApplicationOrder,9156469461,ApplicationCanceled,1.0,NaN,None,None,None,None,NaN,NaN,NaN,新客
3,2019-11-24,P2G&Jet,Jp201911240437,NewApplicationOrder,9656722820,AutoReject,NaN,NaN,None,None,None,None,NaN,NaN,NaN,新客
4,2019-11-24,P2G&Jet,Jp201911240452,NewApplicationOrder,9997970281,AuditingPassed,1.0,1.0,LoanSuccess,3000.0,2019-12-08,2019-12-08,0.0,0.0,0.0,新客


In [21]:
df = load_data_from_pickle(file_path, 'df.pkl')
df["新老标签"] = df["order_type"].map(lambda s: "新客" if s in ["RepeatApplicationOrder", "NewApplicationOrder"] else "老客")
df_result = df.groupby(["apply_date", "新老标签", "product_name"]).agg(
    {"order_no": "count", "jstg": "sum",  "rstg": "sum", "loan_flow_status": "count",
     "loan_principal": "sum", "fdq": "sum", "dpd7": "sum"})
df_result["机审通过率"] = df_result["jstg"]/df_result["order_no"]
df_result["人审通过率"] = df_result["RSTG"]/df_result["jstg"]
df_result["放款通过率"] = df_result["loan_flow_status"]/df_result["order_no"]
df_result["FDQ_%"] = df_result["FDQ"]/df_result["loan_flow_status"]
df_result["DPD7_%"] = df_result["DPD7"]/df_result["loan_flow_status"]
df_result.rename(columns={"order_no": "申请笔数", "loan_flow_status": "放款笔数", "loan_principal": "放款金额"}, inplace=True)
df_result = df_result[["申请笔数", "机审通过率", "人审通过率", "放款笔数", "放款通过率", "放款金额", "FDQ", "FDQ_%", "DPD7", "DPD7_%"]].reset_index()
print(df_result)
# for name in ["P2G&Jet", "CNJ"]:
#     for type in ["新客", "老客"]:
#         df_r = df_result.loc[(df_result["product_name"] == name) & (df_result["新老标签"] == type)]
#         df_r.reset_index(drop=True, inplace=True)
#         df_r.drop(columns=["新老标签", "product_name"], axis=1, inplace=True)
#         if len(df_r):
#             df_r.to_excel(writer, sheet_name="%s_%s" % (name, type))
# # df_result.to_excel("运营指标.xlsx")
# writer.save()

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


KeyError: 'JSTG'